In [1]:
# test - tree construction
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print

urdf_file = "./assets/kuka_iiwa.urdf"
root, links, robot = rmp_tree_from_urdf(urdf_file)

rmp_tree_print(root)

q = np.array([1,2,3,4,5,6,7])
dq = np.array([-1,-2,-3,-4,-5,-6,-7])

root.set_root_state(q,dq)
root.pushforward()


`- root
   |- proj_q_lbr_iiwa_link_1
   |  `- lbr_iiwa_link_1
   |- proj_q_lbr_iiwa_link_2
   |  `- lbr_iiwa_link_2
   |- proj_q_lbr_iiwa_link_3
   |  `- lbr_iiwa_link_3
   |- proj_q_lbr_iiwa_link_4
   |  `- lbr_iiwa_link_4
   |- proj_q_lbr_iiwa_link_5
   |  `- lbr_iiwa_link_5
   |- proj_q_lbr_iiwa_link_6
   |  `- lbr_iiwa_link_6
   `- proj_q_lbr_iiwa_link_7
      `- lbr_iiwa_link_7
root: pushforward
proj_q_lbr_iiwa_link_1: pushforward
proj_q_lbr_iiwa_link_1: x = [1.], dx = [-1.]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.     0.     0.1575 1.     0.     0.    ], dx = [ 0.  0.  0.  0.  0. -1.]
proj_q_lbr_iiwa_link_2: pushforward
proj_q_lbr_iiwa_link_2: x = [1. 2.], dx = [-1. -2.]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36       -1.14159265 -1.         -1.57079633], dx = [ 0.          0.          0.          1.68294197 -1.08060461 -1.        ]
proj_q_lbr_iiwa_link_3: pushforward
proj_q_lbr_iiwa_link_3: x = [1. 2. 3.], dx = [-1. -2. -3.]
lb

In [4]:
import numpy as np
from rmpflow.rmp.rbd_rmp import *
import eigen as e
import rbdyn as rbd
import sva as s
from rbdyn.parsers import *

from scipy.spatial.transform import Rotation as R

ax, ay, az = math.pi/2, math.pi/3, math.pi/4

# euler zyx
rot_sv = s.RotZ(ax)*s.RotY(ay)*s.RotX(az)
rot = rot_sv.transpose()
r = R.from_matrix(np.array(rot))
print("Expected ax, ay, az = {}, {}, {}".format(ax,ay,az))
print(r.as_euler('zyx', degrees=False))


Expected ax, ay, az = 1.5707963267948966, 1.0471975511965976, 0.7853981633974483
[1.57079633 1.04719755 0.78539816]


In [1]:
import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, PositionProjection

root, links = rmp_tree_from_urdf("./assets/kuka_iiwa.urdf")

# link5_pos = PositionProjection("link5_pos", links[''])
for key, value in links.items():
    print(key, " : ", value.name, ", ", value.parent.name, ", ", value.parent.param_map)


lbr_iiwa_link_1  :  lbr_iiwa_link_1 ,  proj_lbr_iiwa_link_1 ,  [1 0 0 0 0 0 0]
lbr_iiwa_link_2  :  lbr_iiwa_link_2 ,  proj_lbr_iiwa_link_2 ,  [1 1 0 0 0 0 0]
lbr_iiwa_link_3  :  lbr_iiwa_link_3 ,  proj_lbr_iiwa_link_3 ,  [1 1 1 0 0 0 0]
lbr_iiwa_link_4  :  lbr_iiwa_link_4 ,  proj_lbr_iiwa_link_4 ,  [1 1 1 1 0 0 0]
lbr_iiwa_link_5  :  lbr_iiwa_link_5 ,  proj_lbr_iiwa_link_5 ,  [1 1 1 1 1 0 0]
lbr_iiwa_link_6  :  lbr_iiwa_link_6 ,  proj_lbr_iiwa_link_6 ,  [1 1 1 1 1 1 0]
lbr_iiwa_link_7  :  lbr_iiwa_link_7 ,  proj_lbr_iiwa_link_7 ,  [1 1 1 1 1 1 1]


In [1]:
import pybullet as b
import time
import pybullet_data

import numpy as np
from rmpflow.rmp.rbd_rmp import rmp_tree_from_urdf, rmp_tree_print, node_array, ProjectionNode, PositionProjection, FrameProjection
import rmpflow.rmp.rmp_leaf as leaves

# constants
DT = 0.001 #s
urdf_file = "./assets/kuka_iiwa.urdf"

# import rmp tree
root, links, robot = rmp_tree_from_urdf(urdf_file)

# attach frame projector and position attractor
proj_node = PositionProjection("proj_xp_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
target_pos = np.array([0.5,0.5,0.5])
atrc = leaves.PosAttractorUni("pos_attractor", proj_node, target_pos.transpose(), gain=1600, eta=80)

# attach frame projector and orientation projector
# proj_node = RotationProjection("proj_xq_lbr_iiwa_link_7", links["lbr_iiwa_link_7"])
# target_pos = np.array([0.5,0.5,0.5])
# atrc = leaves.GoalAttractorUni("pos_attractor", proj_node, target_pos.transpose(), gain=1600, eta=80)

# tree printing
# rmp_tree_print(root)

q = np.random.rand(7)
dq = np.random.rand(7)

# if isinstance(proj_node, PositionProjection):
#     print("asdfadf")

root.set_root_state(q,dq)
root.pushforward()
root.pullback()


# # setup physics
# physicsClient = b.connect(b.GUI)
# b.setAdditionalSearchPath(pybullet_data.getDataPath())
# b.setGravity(0,0,-9.81)
# b.setRealTimeSimulation(0)
# b.setTimeStep(DT)

# # import robot
# plane_id = b.loadURDF("plane.urdf")
# start_pos = [0,0,0]
# start_ori = b.getQuaternionFromEuler([0,0,0])
# robot_id = b.loadURDF(urdf_file,start_pos, start_ori, flags=(b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))
# n_dof = b.getNumJoints(robot_id)
# joints_id = range(n_dof) 

# # setup configuration
# b.setJointMotorControlArray(robot_id, joints_id, b.VELOCITY_CONTROL, forces=[0] * n_dof)

# # simulate
# for i in range(0, 10000):
#     # read state feedback
#     joint_states = b.getJointStates(robot_id, joints_id)
#     q = np.array([joint_states[i][0] for i in joints_id])
#     dq = np.array([joint_states[i][1] for i in joints_id])
#     ddq, mass_matrix, h = root.solve(q,dq)

#     # compute torque and dynamics
#     tau = (mass_matrix.dot(ddq) + h).flatten()

#     # command system:
#     for i in joints_id:
#         b.setJointMotorControl2(robot_id, i, b.TORQUE_CONTROL, force=tau[i])

#     # sleep
#     b.stepSimulation()
#     time.sleep(DT)

# # disconnect
# b.disconnect()

proj_q_lbr_iiwa_link_1: pushforward
proj_q_lbr_iiwa_link_1: x = [0.06464992], dx = [0.98728733]
lbr_iiwa_link_1: pushforward
lbr_iiwa_link_1: x = [0.         0.         0.1575     0.         0.         0.03231933
 0.99947759], dx = [0.         0.         0.         0.         0.         0.98728733]
proj_q_lbr_iiwa_link_2: pushforward
proj_q_lbr_iiwa_link_2: x = [0.06464992 0.19790701], dx = [0.98728733 0.26896334]
lbr_iiwa_link_2: pushforward
lbr_iiwa_link_2: x = [ 0.          0.          0.36        0.04707864  0.70553781  0.70102237
 -0.09256149], dx = [ 0.          0.          0.         -0.01737635  0.26840145  0.98728733]
proj_q_lbr_iiwa_link_3: pushforward
proj_q_lbr_iiwa_link_3: x = [0.06464992 0.19790701 0.70857118], dx = [0.98728733 0.26896334 0.0570086 ]
lbr_iiwa_link_3: pushforward
lbr_iiwa_link_3: x = [0.04012431 0.00259765 0.56050821 0.03126049 0.09371585 0.37520681
 0.92166153], dx = [ 0.05125206  0.04309832 -0.01081456 -0.00619087  0.2691256   1.04318314]
proj_q_lbr_iiwa